<a href="https://colab.research.google.com/github/Gretarobur/pyqb-20220714/blob/main/Pipeline_foci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os
from collections import Counter
!pip install xlsxwriter
from tqdm import tqdm
import time
import os
import subprocess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
directory = "/content/drive/MyDrive/Analysis"
files = os.listdir(directory)

fields = []
for subfolder in files:
  fields.append(subfolder)

directory_analysis = []
for subfolder in files:
  directory_analysis.append(directory + "/" + subfolder)

In [ ]:
field = []
for subfolder in directory_analysis:
  field.append(subfolder)

In [ ]:
def la_distanza(il_pallino, il_nucleo):
  return math.sqrt((il_nucleo[0] - il_pallino[0])**2 + (il_nucleo[1] - il_pallino[1])**2)

def distance_3D(il_pallino, nuclei_data):
  return math.sqrt((nuclei_data[1] - il_pallino[1])**2 + (nuclei_data[2] - il_pallino[2])**2 + (nuclei_data[3] - il_pallino[3])**2)

def volume_to_radius(volume):
  return((3*volume)/(4* math.pi))**(1/3)

In [ ]:
def easy_plot (dict_merged,parametro, doppioni = False):
  easy_plot = None
  keys = []
  for key in dict_merged.keys():
    easy_plot = pd.concat([easy_plot,dict_merged[key][parametro]],axis=1)
    if doppioni == True:
      easy_plot = easy_plot.drop_duplicates()
      keys.append(key)
  easy_plot.columns = keys
  easy_plot.to_excel(xlsx_file_summary, sheet_name = parametro)
  return None

In [ ]:
#@title !!! Voxel Size: Inserire la dimensione dei pixel in X/Y e Z (Info dell'immagine) !!!
XY_pixel = 0.0801 #@param {type:"raw"}
Z_pixel = 0.4196 #@param {type:"raw"}

voxel_size = XY_pixel * XY_pixel * Z_pixel

In [ ]:
retcode = subprocess.call(['echo', 'foo'],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT)


dict_merged = {}
list_merged = []
#directory_analysis è una lista con i percorsi dei file di tutte le cartelle
  #in Analysis su google drive
for experiment in tqdm(directory_analysis):
  #questo sotto serve per i dizionari per fare il summary
  list_merged.append(experiment.split("/")[-1])
  #crea una lista con tutti i file nella sotto
  #cartella directory_analysis (dovrebbero esserci solo i file csv)
  csv = os.listdir(experiment)

  #crea il percorso per il file excel
  xlsx_file_path = experiment + "/" + experiment.split("/")[-1] + ".xlsx"
  print(xlsx_file_path)

  #sorta la lista per avere sempre lo stesso ordine
  csv.sort()
  print(experiment)

  #Percorsi dei file CSV
  #percorso file MASCHERE del csv - nucleoplasma
  nome_file_nuclei = experiment + "/" + csv[1]
  #percorso file MASCHERE del csv - foci
  nome_file_foci = experiment + "/" + csv[3]
  #percorso file MASCHERE del csv - nucleo
  nome_file_total = experiment + "/" + csv[2]

  #CANALE_1
  #percorso file QUANTIFICA del csv - nucleoplasma
  nome_file_quantifiche_nucleoplasma_canale_1 = experiment + "/" + csv[1]
  #percorso file QUANTIFICA del csv - nucleo
  nome_file_quantifiche_total_canale_1 = experiment + "/" + csv[1]
  #percorso file QUANTIFICA del csv - nucleolo
  nome_file_quantifiche_nucleolo_canale_1 = experiment + "/" + csv[3] #foci

  #CANALE_2
  #percorso file QUANTIFICA del csv - nucleoplasma
  nome_file_quantifiche_nucleoplasma_canale_2 = experiment + "/" + csv[2]
  #percorso file QUANTIFICA del csv - nucleo
  nome_file_quantifiche_total_canale_2 = experiment + "/" + csv[2]
  #percorso file QUANTIFICA del csv - nucleolo
  nome_file_quantifiche_nucleolo_canale_2 = experiment + "/" + csv[3] #foci
#------------------------------------------------------------------------------
  #Generazione dei file CSV e liste con tutto ciò che serve

  #INFO e DF sul nucleo TOTALE
  nuclei_data = []
  #Lista indici nuclei
  N_nuclei = pd.read_csv(nome_file_total)['Nb'].tolist()
  #lista coordinata x del centroide
  x_c_nuclei = pd.read_csv(nome_file_total)['CX (unit)'].tolist()
  #lista coordinata y del centroide
  y_c_nuclei = pd.read_csv(nome_file_total)['CY (unit)'].tolist()
  #lista coordinata z del centroide
  z_c_nuclei = pd.read_csv(nome_file_total)['CZ (unit)'].tolist()
  #lista volume del nucleo
  volume_nuclei = pd.read_csv(nome_file_total)['Vol (unit)'].tolist()
  #Lista indici delle fluorescenze canale_1
  N_nuclei_canale = pd.read_csv(
                    nome_file_quantifiche_total_canale_1)['Nb'].tolist()
  #Lista fluorescenze canale_1
  MFI_ch1 = pd.read_csv(
            nome_file_quantifiche_total_canale_1)['Mean'].tolist()
  #Lista indici delle fluorescenze canale_2
  N_nuclei_canale = pd.read_csv(
                    nome_file_quantifiche_total_canale_2)['Nb'].tolist()
  #Lista fluorescenze canale_2
  MFI_ch2 = pd.read_csv(nome_file_quantifiche_total_canale_2)['Mean'].tolist()
  #Integrated intensity canale_1
  IntDen_ch1 = pd.read_csv(nome_file_quantifiche_total_canale_1)['IntDen'].tolist()
  #Integrated intensity canale_2
  IntDen_ch2 = pd.read_csv(nome_file_quantifiche_total_canale_2)['IntDen'].tolist()
  #Standard_deviation of the signal
  ch1_SD = pd.read_csv(nome_file_quantifiche_total_canale_1)['Mean'].tolist()


  #Lista di liste con tutte le info dei nuclei
  nuclei_data = [(N_nuclei[i], x_c_nuclei[i], y_c_nuclei[i], z_c_nuclei[i],
                  volume_nuclei[i], MFI_ch1[i], MFI_ch2[i], IntDen_ch1[i], IntDen_ch2[i], ch1_SD[i])
                  for i in range(0, len(x_c_nuclei))]
  #DataFrame pandas dei nuclei
  Nuclei_DF = pd.DataFrame (nuclei_data, columns =
   ["Nb", "Cx", "Cy","Cz","Volume_unit","MFI_ch1","MFI_ch2","IntDen_ch1","IntDen_ch2","SD"])
#------------------------------------------------------------------------------
  #INFO e DF sul nucleoplasma

  nucleoplasm_data = []
  #Lista indici nucleoplasma
  N_nuclei = pd.read_csv(nome_file_nuclei)['Nb'].tolist()
  #lista coordinata x del centroide
  x_c_nuclei = pd.read_csv(nome_file_nuclei)['CX (unit)'].tolist()
  #lista coordinata y del centroide
  y_c_nuclei = pd.read_csv(nome_file_nuclei)['CY (unit)'].tolist()
  #lista coordinata z del centroide
  z_c_nuclei = pd.read_csv(nome_file_nuclei)['CZ (unit)'].tolist()
  #lista volume del nucleoplasma
  volume_nuclei = pd.read_csv(nome_file_nuclei)['Vol (unit)'].tolist()
  #Lista indici delle fluorescenze nucleoplasma canale_1
  N_nuclei_canale = pd.read_csv(
                    nome_file_quantifiche_nucleoplasma_canale_1)['Nb'].tolist()
  #Lista fluorescenze nucleoplasma canale_1
  MFI_ch1 = pd.read_csv(
            nome_file_quantifiche_nucleoplasma_canale_1)['Mean'].tolist()
  #Lista indici delle fluorescenze nucleoplasma canale_2
  N_nuclei_canale = pd.read_csv(
                    nome_file_quantifiche_nucleoplasma_canale_2)['Nb'].tolist()
  #Lista fluorescenze nucleoplasma canale_2
  MFI_ch2 = pd.read_csv(
            nome_file_quantifiche_nucleoplasma_canale_2)['Mean'].tolist()
  #Integrated intensity ch1
  IntDen_ch1 = pd.read_csv(
            nome_file_quantifiche_nucleoplasma_canale_1)['IntDen'].tolist()
  #Integrated intensity ch2
  IntDen_ch2 = pd.read_csv(
            nome_file_quantifiche_nucleoplasma_canale_2)['IntDen'].tolist()


  #Lista di liste con tutte le info del nucleoplasma
  nucleoplasm_data = [(N_nuclei[i], x_c_nuclei[i], y_c_nuclei[i], z_c_nuclei[i],
                  volume_nuclei[i], MFI_ch1[i], MFI_ch2[i], IntDen_ch1[i], IntDen_ch2[i])
                  for i in range(0, len(x_c_nuclei))]
  #DataFrame pandas dei nucleoplasma
  Nucleoplasmi_DF = pd.DataFrame (nucleoplasm_data, columns =
              ["Nb", "Cx", "Cy","Cz","Volume_unit","MFI_ch1","MFI_ch2","IntDen_ch1","IntDen_ch2"])
#------------------------------------------------------------------------------
  #INFO e DF sui foci
  foci_data = []

  #lista indici foci
  N_foci = pd.read_csv(nome_file_foci)['Peak #'].tolist()
  #lista coordinata x del centroide
  x_c_foci = pd.read_csv(nome_file_foci)['X'].tolist()
  x_c_foci = [i * XY_pixel for i in x_c_foci]
  #lista coordinata y del centroide
  y_c_foci = pd.read_csv(nome_file_foci)['Y'].tolist()
  y_c_foci = [i * XY_pixel for i in y_c_foci]
  #lista coordinata z del centroide
  z_c_foci = pd.read_csv(nome_file_foci)['Z'].tolist()
  z_c_foci = [i * Z_pixel for i in z_c_foci]
  #lista volumi dei foci
  volume_foci = pd.read_csv(nome_file_foci)['Size'].tolist()
  volume_foci = [i * voxel_size for i in volume_foci]
  #fluorescenza foci
  MFI_ch1_foci = pd.read_csv(
                    nome_file_quantifiche_nucleolo_canale_1)['Av'].tolist()
  #Integrated intensity ch1
  IntDen_ch1 = pd.read_csv(
            nome_file_quantifiche_nucleolo_canale_1)['Total'].tolist()

  #Lista di liste con tutte le info sui foci
  foci_data = [(N_foci[i], x_c_foci[i], y_c_foci[i], z_c_foci[i],
                  volume_foci[i], MFI_ch1_foci[i], IntDen_ch1[i])
                  for i in range(0, len(x_c_foci))]
  #DataFrame pandas dei foci
  foci_DF = pd.DataFrame (foci_data, columns =
                ["Nb", "Cx", "Cy","Cz","Volume_unit","Mean_ch1","IntDen_ch1"])

  #Rimuovo i foci troppo grandi > 5um^3
  foci_DF.drop(foci_DF[foci_DF['Volume_unit'] >= 5].index, inplace = True)
  #Rimuovo i foci troppo poco intensi
  #foci_DF.drop(foci_DF[foci_DF['Mean_ch1'] <=4000 ].index, inplace = True)
#-------------------------------------------------------------------------------
  #Associazione nucleolo -> nucleo
  #Creo un dizionario per
  nuclei = [Nucleoplasmi_DF,Nuclei_DF]
  #0 per nucleoplasmi 1 per nuclei totali
  touse = 1

  conta_dxn = {}
  nuclei_index = []
  min_distance = []
  Nuclei_index_touse = nuclei[touse].Nb.tolist()
  #iterazione sui foci
  for i, row_D in foci_DF.iterrows():
    distances = []
    for k, row_N in nuclei[touse].iterrows():
      nucleolus_distance = distance_3D (row_D, row_N)
      distances.append(nucleolus_distance)

    index_min = distances.index(min(distances))
    min_distance.append(min(distances))
    nuclei_index.append(Nuclei_index_touse[index_min])

  foci_DF["Centroid_distance"] = min_distance
  foci_DF["Nucleoplasm_ID"] = nuclei_index
  foci_DF["Nucleoplasm_ID"] = foci_DF["Nucleoplasm_ID"]



  conta_dxn = Counter(foci_DF["Nucleoplasm_ID"])
  keys = list(conta_dxn.keys())
  keys.sort()
  sorted_conta_dxn = {i: conta_dxn[i] for i in keys}

  key_list = list(sorted_conta_dxn.keys())
  value_list = list(sorted_conta_dxn.values())
  Dots_Nucleus = pd.DataFrame(key_list, columns = ["Nucleo"])
  Dots_Nucleus["Dots"] = value_list

  #-----------------------------------------------------------------------------
  #Calcolo partition ratio nucleolo/nucleoplasma
  #DataFrame.iloc[i] prende la i-esima riga di un dataframe

  #partition ratio del primo canale
  pr_ch1 = []
  for i in range (0, len(foci_DF)):
    pr_ch1.append(foci_DF["Mean_ch1"].iloc[i] / nuclei[touse].iloc[int(foci_DF.iloc[i]["Nucleoplasm_ID"])]["MFI_ch1"])
  foci_DF["PR_ch1"] = pr_ch1

  #-----------------------------------------------------------------------------

  #----------------------------------------------------------------------------
  #Preparazione file excel finali
  #crea il file excel
  xlsx_file = pd.ExcelWriter(xlsx_file_path, engine='xlsxwriter')

  Merged = pd.merge(nuclei[touse], foci_DF, left_on='Nb', right_on='Nucleoplasm_ID')
  Merged["%IntDen"] = Merged["IntDen_ch1_y"]/Merged["IntDen_ch1_x"]
  Merged["Distanza norm."] = Merged["Centroid_distance"]/volume_to_radius(Merged["Volume_unit_x"])*100
  Merged["nuclei_id"] = None

  set_nuclei = list(set(Merged["Nb_x"].to_list()))
  volume_perc = []
  volume_perc_flat = []
  for i in set_nuclei:
    a = Merged.loc[(Merged["Nb_x"] == i)]["Volume_unit_y"].sum()/ Merged.loc[(Merged["Nb_x"]==i)]["Volume_unit_x"]*100
    a = a.tolist()
    volume_perc_flat.append(a)

  volume_perc_flat = [item for sublist in volume_perc_flat for item in sublist]
  Merged["%vol_flat"] = volume_perc_flat
  Merged.drop(Merged[Merged["%vol_flat"]>=90].index, inplace = True)
  set_vol_perc = list(set(Merged["%vol_flat"].tolist()))

  Merged["%vol"] = None
  for i in range(0, len(set_vol_perc)):
    Merged["%vol"][i] = set_vol_perc[i]

  mfich1 = list(set(Merged["MFI_ch1"].tolist()))
  Merged["mfinucleo"] = None
  for i in range(0, len(mfich1)):
    Merged["mfinucleo"][i] = mfich1[i]

  Merged["foci_numbers"] = None
  for i in range(0, len(list(conta_dxn.keys()))):
    Merged["nuclei_id"][i] = list(conta_dxn.keys())[i]
    Merged["foci_numbers"][i] = list(conta_dxn.values())[i]
  Merged.to_excel(xlsx_file, sheet_name = "analysis")
  Merged["SD/mean"] = Merged["SD"]/Merged["MFI_ch1"]

  counter = 0
  for i in Merged["foci_numbers"]:
    if i is None:
      counter = counter + 0
    else:
      counter = counter + 1
  negative_nuclei = len(Nuclei_DF) - counter
  positive_nuclei = counter
  total_nuclei = len(Nuclei_DF)
  fraction = positive_nuclei/total_nuclei
  print(negative_nuclei)
  print(positive_nuclei)
  cell_positive = ["positive_nuclei", positive_nuclei ,"negative_nuclei", negative_nuclei, "total_nuclei", total_nuclei, "fraction", fraction]

  Merged["cell_positive"] = None
  for i in range (0, len(cell_positive)):
    Merged["cell_positive"][i] = cell_positive[i]

  #n_positive = len(Merged[])

  list_merged.append(Merged)
  xlsx_file.save()

dict_merged = {}
for i in range(0,len(list_merged),2):
  dict_merged[list_merged[i]] = list_merged[i+1]

xlsx_file_path_summary = "/content/drive/MyDrive/Analysis/" + "summary.xlsx"
xlsx_file_summary = pd.ExcelWriter(xlsx_file_path_summary, engine='xlsxwriter')


easy_plot_dn = None
keys = []
for key in dict_merged.keys():
  easy_plot_dn = pd.concat([easy_plot_dn, dict_merged[key]["Distanza norm."]], axis = 1)
  keys.append(key)
easy_plot_dn.columns = keys
easy_plot_dn.to_excel(xlsx_file_summary, sheet_name = "distanza_normalizzata")

easy_plot_vol = None
keys = []
for key in dict_merged.keys():
  easy_plot_vol = pd.concat([easy_plot_vol, dict_merged[key]["Volume_unit_y"]], axis = 1)
  keys.append(key)
easy_plot_vol.columns = keys
easy_plot_vol.to_excel(xlsx_file_summary, sheet_name = "Volumi")

easy_plot_nnuc = None
keys = []
for key in dict_merged.keys():
  easy_plot_nnuc = pd.concat([easy_plot_nnuc, dict_merged[key]["foci_numbers"]], axis = 1)
  keys.append(key)
easy_plot_nnuc.columns = keys
easy_plot_nnuc.to_excel(xlsx_file_summary, sheet_name = "Numero foci")

easy_plot_prch1 = None
keys = []
for key in dict_merged.keys():
  easy_plot_prch1 = pd.concat([easy_plot_prch1, dict_merged[key]["PR_ch1"]], axis = 1)
  keys.append(key)
easy_plot_prch1.columns = keys
easy_plot_prch1.to_excel(xlsx_file_summary, sheet_name = "PR_ch1")

easy_plot_mfich1 = None
keys = []
for key in dict_merged.keys():
  easy_plot_mfich1 = pd.concat([easy_plot_mfich1, dict_merged[key]["MFI_ch1"]], axis = 1)
  easy_plot_mfich1 = easy_plot_mfich1.drop_duplicates()
  keys.append(key)
easy_plot_mfich1.columns = keys
easy_plot_mfich1.to_excel(xlsx_file_summary, sheet_name = "MFI_ch1")

easy_plot_intden = None
keys = []
for key in dict_merged.keys():
  easy_plot_intden = pd.concat([easy_plot_intden, dict_merged[key]["%IntDen"]], axis = 1)
  keys.append(key)
easy_plot_intden.columns = keys
easy_plot_intden.to_excel(xlsx_file_summary, sheet_name = "%IntDen")

easy_plot_sd_mean = None
keys = []
for key in dict_merged.keys():
  easy_plot_sd_mean = pd.concat([easy_plot_sd_mean, dict_merged[key]["SD/mean"]], axis = 1)
  easy_plot_sd_mean = easy_plot_sd_mean.drop_duplicates()
  keys.append(key)
easy_plot_sd_mean.columns = keys
easy_plot_sd_mean.to_excel(xlsx_file_summary, sheet_name = "SD_mean")

easy_plot_mfi_foci = None
keys = []
for key in dict_merged.keys():
  easy_plot_mfi_foci = pd.concat([easy_plot_mfi_foci, dict_merged[key]["Mean_ch1"]], axis = 1)
  easy_plot_mfi_foci = easy_plot_mfi_foci.drop_duplicates()
  keys.append(key)
easy_plot_mfi_foci.columns = keys
easy_plot_mfi_foci.to_excel(xlsx_file_summary, sheet_name = "mfi_foci")

easy_plot_negative = None
keys = []
for key in dict_merged.keys():
  easy_plot_negative = pd.concat([easy_plot_negative, dict_merged[key]["cell_positive"]], axis = 1)
  easy_plot_negative = easy_plot_negative.drop_duplicates()
  keys.append(key)
easy_plot_negative.columns = keys
easy_plot_negative.to_excel(xlsx_file_summary, sheet_name = "cell_positive")

easy_plot_percvol = None
keys = []
for key in dict_merged.keys():
  easy_plot_percvol = pd.concat([easy_plot_percvol, dict_merged[key]["%vol"]], axis = 1)
  keys.append(key)
easy_plot_percvol.columns = keys
easy_plot_percvol.to_excel(xlsx_file_summary, sheet_name = "%vol")


xlsx_file_summary.save()

  0%|          | 0/10 [00:00<?, ?it/s]

/content/drive/MyDrive/Analysis/SERIE_10/SERIE_10.xlsx
/content/drive/MyDrive/Analysis/SERIE_10


<ipython-input-8-c4738ff00f15>:242: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["%vol"][i] = set_vol_perc[i]
<ipython-input-8-c4738ff00f15>:247: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["mfinucleo"][i] = mfich1[i]
<ipython-input-8-c4738ff00f15>:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["nuclei_id"][i] = list(conta_dxn.keys())[i]
<ipython-input-8-c4738ff00f15>:252: SettingWithCopyWarning: 
A value is

16
37
/content/drive/MyDrive/Analysis/SERIE_01/SERIE_01.xlsx
/content/drive/MyDrive/Analysis/SERIE_01


<ipython-input-8-c4738ff00f15>:242: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["%vol"][i] = set_vol_perc[i]
<ipython-input-8-c4738ff00f15>:247: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["mfinucleo"][i] = mfich1[i]
<ipython-input-8-c4738ff00f15>:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["nuclei_id"][i] = list(conta_dxn.keys())[i]
<ipython-input-8-c4738ff00f15>:252: SettingWithCopyWarning: 
A value is

...

<ipython-input-8-c4738ff00f15>:242: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["%vol"][i] = set_vol_perc[i]
<ipython-input-8-c4738ff00f15>:247: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["mfinucleo"][i] = mfich1[i]
<ipython-input-8-c4738ff00f15>:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["nuclei_id"][i] = list(conta_dxn.keys())[i]
<ipython-input-8-c4738ff00f15>:252: SettingWithCopyWarning: 
A value is

1
25
/content/drive/MyDrive/Analysis/SERIE_03/SERIE_03.xlsx
/content/drive/MyDrive/Analysis/SERIE_03


<ipython-input-8-c4738ff00f15>:242: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["%vol"][i] = set_vol_perc[i]
<ipython-input-8-c4738ff00f15>:247: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["mfinucleo"][i] = mfich1[i]
<ipython-input-8-c4738ff00f15>:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merged["nuclei_id"][i] = list(conta_dxn.keys())[i]
<ipython-input-8-c4738ff00f15>:252: SettingWithCopyWarning: 
A value is

6
43



<ipython-input-8-c4738ff00f15>:372: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  xlsx_file_summary.save()


In [ ]:
nuclei_data

[(0,
  3.437993427663201,
  2.5990144725971507,
  2.0074526212629933,
  32.70723117904361,
  313.50419476887646,
  680.0043592696167,
  3811584.0,
  8267493.0,
  313.50419476887646),
 (1,
  9.697822704319693,
  0.1580136576645741,
  1.5183288106745436,
  1.318189116444429,
  211.64897959183676,
  1224.8857142857144,
  103708.0,
  600194.0,
  211.64897959183676),
 (2,
  43.424442634100906,
  2.609278626753863,
  2.1493445624483862,
  36.07264869878234,
  1294.174733388023,
  4479.440152136624,
  17353589.0,
  60064813.0,
  1294.174733388023),
 (3,
  51.60953259725625,
  0.7967284671951077,
  2.705683119446336,
  14.790619922880552,
  863.580210985813,
  3980.824299745362,
  4747964.0,
  21886572.0,
  863.580210985813),
 (4,
  11.437469400204542,
  6.82291221102437,
  1.6211591980242197,
  34.13571775217012,
  940.5109149657184,
  3702.166522184569,
  11934143.0,
  46976791.0,
  940.5109149657184),
 (5,
  62.894998514841966,
  7.284868064841908,
  2.1841200729978536,
  34.11419629720776,

In [ ]:
len(Merged["foci_numbers"])

counter = 0
for i in Merged["foci_numbers"]:
  if i is None:
    counter = counter + 0
  else:
    counter = counter + 1
negative_nuclei = len(Nuclei_DF) - counter
positive_nuclei = counter
total_nuclei = len(Nuclei_DF)
fraction = positive_nuclei/total_nuclei
print(negative_nuclei)
print(positive_nuclei)

1
33


In [ ]:
a = [1,2,3,4,5,6]

for i in tqdm(a):
  print(i)
  time.sleep(0.5)

  0%|          | 0/6 [00:00<?, ?it/s]

1


 17%|█▋        | 1/6 [00:00<00:02,  1.99it/s]

2


 33%|███▎      | 2/6 [00:01<00:02,  1.99it/s]

...

 67%|██████▋   | 4/6 [00:02<00:01,  1.99it/s]

5


 83%|████████▎ | 5/6 [00:02<00:00,  1.99it/s]

6


100%|██████████| 6/6 [00:03<00:00,  1.99it/s]


In [ ]:
  conta_dxn = {}
  nuclei_index = []
  min_distance = []
  Nuclei_index_touse = Nuclei_DF.Nb.tolist()
  #iterazione sui nucleoli
  for i, row_D in Nucleoli_DF.iterrows():
    distances = []
    for k, row_N in Nucleoplasmi_DF.iterrows():
      nucleolus_distance = distance_3D (row_D, row_N)
      distances.append(nucleolus_distance)

    index_min = distances.index(min(distances))
    min_distance.append(min(distances))
    nuclei_index.append(Nuclei_index_touse[index_min])

NameError: ignored

In [ ]:
Dots_nucleus

In [ ]:
experiment + "/" + csv[6]

In [ ]:
nome_file_quantifiche_nucleoplasma_canale_2

In [ ]:
list_merged = []
dict_merged = {}
for experiment in directory_analysis:

  # Lista dei centroidi dei nuclei
  # Nuclei = [(X_centroid, Y_centroid, Z_centroid, X_max, X_min, Y_max, Y_min, Z_max, Z_min, volume)]
  #nuclei = [("N_nuclei", "x_c_nuclei", "y_c_nuclei", "z_c_nuclei", "volume_nuclei", "X_max", "X_min", "Y_max", "Y_min", "Z_max", "Z_min")]
  list_merged.append(experiment.split("/")[-1])

  csv = os.listdir(experiment)
  file_path = experiment + "/" + csv[1]
  csv.sort()
  print(experiment)

  xlsx_file_path = experiment + "/" + experiment.split("/")[-1] + ".xlsx"
  print(xlsx_file_path)
  xlsx_file = pd.ExcelWriter(xlsx_file_path, engine='xlsxwriter')
  print(csv)

  #Da aggiungere i file dei nucleoli e nuclei-nucleoli sia in M che in Q


  nome_file_nuclei = experiment + "/" + csv[0]
  nome_file_nucleoli = experiment + "/" + csv[1]
  nome_file_total = experiment + "/" + csv[2]


  nome_file_quantifiche_nucleo_canale = experiment + "/" + csv[5]
  nome_file_quantifiche_total_canale = experiment + "/" + csv[3]
  nome_file_quantifiche_nucleolo_canale = experiment + "/" + csv[7]

  nuclei_data = []

  N_nuclei = pd.read_csv(nome_file_nuclei)['Nb'].tolist()
  x_c_nuclei = pd.read_csv(nome_file_nuclei)['CX (unit)'].tolist()
  y_c_nuclei = pd.read_csv(nome_file_nuclei)['CY (unit)'].tolist()
  z_c_nuclei = pd.read_csv(nome_file_nuclei)['CZ (unit)'].tolist()
  volume_nuclei = pd.read_csv(nome_file_nuclei)['Vol (unit)'].tolist()

  N_nuclei_canale = pd.read_csv(nome_file_quantifiche_nucleo_canale)['Nb'].tolist()
  MFI = pd.read_csv(nome_file_quantifiche_nucleo_canale)['Mean'].tolist()

  channel_data_nuclei = [(N_nuclei_canale[i], MFI[i]) for i in range(0, len(x_c_nuclei))]

  #X_max = pd.read_csv(nome_file_nuclei)['Xmax (pix)'].tolist()
  #X_min = pd.read_csv(nome_file_nuclei)['Xmin (pix)'].tolist()
  #Y_max = pd.read_csv(nome_file_nuclei)['Ymax (pix)'].tolist()
  #Y_min = pd.read_csv(nome_file_nuclei)['Ymin (pix)'].tolist()
  #Z_max = pd.read_csv(nome_file_nuclei)['Zmax (pix)'].tolist()
  #Z_min = pd.read_csv(nome_file_nuclei)['Zmin (pix)'].tolist()

  #nuclei_data = [(N_nuclei[i], x_c_nuclei[i], y_c_nuclei[i], z_c_nuclei[i], volume_nuclei[i], X_max[i], X_min[i], Y_max[i], Y_min[i], Z_max[i], Z_min[i]) for i in range(0, len(x_c_nuclei))]
  nuclei_data = [(N_nuclei[i], x_c_nuclei[i], y_c_nuclei[i], z_c_nuclei[i], volume_nuclei[i]) for i in range(0, len(x_c_nuclei))]

  nucleoli_data = []

  N_nucleoli = pd.read_csv(nome_file_nucleoli)['Nb'].tolist()
  x_c_nucleoli = pd.read_csv(nome_file_nucleoli)['CX (unit)'].tolist()
  #x_c_nucleoli = [i * XY_pixel for i in x_c_nucleoli]
  y_c_nucleoli = pd.read_csv(nome_file_nucleoli)['CY (unit)'].tolist()
  #y_c_nucleoli = [i * XY_pixel for i in y_c_nucleoli]
  z_c_nucleoli = pd.read_csv(nome_file_nucleoli)['CZ (unit)'].tolist()
  #z_c_nucleoli = [i * Z_pixel for i in z_c_nucleoli]
  volume_nucleoli = pd.read_csv(nome_file_nucleoli)['Vol (unit)'].tolist()
  #volume_nucleoli = [i * voxel_size for i in volume_nucleoli]

  N_nucleoli_canale = pd.read_csv(nome_file_quantifiche_nucleolo_canale)['Nb'].tolist()

  MFI = pd.read_csv(nome_file_quantifiche_nucleolo_canale)['Mean'].tolist()
  channel_data_nucleoli = [(N_nucleoli_canale[i], MFI[i]) for i in range(0, len(x_c_nucleoli))]

  #X_max = pd.read_csv(nome_file_nucleoli)['Xmax (pix)'].tolist()
  #X_min = pd.read_csv(nome_file_nucleoli)['Xmin (pix)'].tolist()
  #Y_max = pd.read_csv(nome_file_nucleoli)['Ymax (pix)'].tolist()
  #Y_min = pd.read_csv(nome_file_nucleoli)['Ymin (pix)'].tolist()
  #Z_max = pd.read_csv(nome_file_nucleoli)['Zmax (pix)'].tolist()
  #Z_min = pd.read_csv(nome_file_nucleoli)['Zmin (pix)'].tolist()

  #nucleoli_data = [(N_nucleoli[i], x_c_nucleoli[i], y_c_nucleoli[i], z_c_nucleoli[i], volume_nucleoli[i], X_max[i], X_min[i], Y_max[i], Y_min[i], Z_max[i], Z_min[i]) for i in range(0, len(x_c_nucleoli))]
  nucleoli_data = [(N_nucleoli[i], x_c_nucleoli[i], y_c_nucleoli[i], z_c_nucleoli[i], volume_nucleoli[i]) for i in range(0, len(x_c_nucleoli))]


  total_data = []

  N_total = pd.read_csv(nome_file_total)['Nb'].tolist()
  x_c_total = pd.read_csv(nome_file_total)['CX (unit)'].tolist()
  y_c_total = pd.read_csv(nome_file_total)['CY (unit)'].tolist()
  z_c_total = pd.read_csv(nome_file_total)['CZ (unit)'].tolist()
  volume_total = pd.read_csv(nome_file_total)['Vol (unit)'].tolist()

  N_total_canale = pd.read_csv(nome_file_quantifiche_total_canale)['Nb'].tolist()
  MFI = pd.read_csv(nome_file_quantifiche_total_canale)['Mean'].tolist()

  channel_data_total = [(N_total_canale[i], MFI[i]) for i in range(0, len(x_c_total))]

  #X_max = pd.read_csv(nome_file_total)['Xmax (pix)'].tolist()
  #X_min = pd.read_csv(nome_file_total)['Xmin (pix)'].tolist()
  #Y_max = pd.read_csv(nome_file_total)['Ymax (pix)'].tolist()
  #Y_min = pd.read_csv(nome_file_total)['Ymin (pix)'].tolist()
  #Z_max = pd.read_csv(nome_file_total)['Zmax (pix)'].tolist()
  #Z_min = pd.read_csv(nome_file_total)['Zmin (pix)'].tolist()

  #total_data = [(N_total[i], x_c_total[i], y_c_total[i], z_c_total[i], volume_total[i], X_max[i], X_min[i], Y_max[i], Y_min[i], Z_max[i], Z_min[i]) for i in range(0, len(x_c_total))]
  total_data = [(N_total[i], x_c_total[i], y_c_total[i], z_c_total[i], volume_total[i]) for i in range(0, len(x_c_total))]


  Nuclei_DF = pd.DataFrame (nuclei_data, columns = ["Nb", "Cx", "Cy","Cz","Volume_unit"])
  Nuclei_DF["Nb"] = Nuclei_DF["Nb"] + 1
  Nuclei_DF['IntDen_Nuclei'] = pd.read_csv(nome_file_quantifiche_nucleo_canale)['IntDen'].tolist()
  #Nuclei_DF.drop(Nuclei_DF[Nuclei_DF['Volume_unit'] <= 80].index, inplace = True)
  Nuclei_DF["Volume_pix"] = Nuclei_DF["Volume_unit"] / voxel_size

  Total_DF = pd.DataFrame (total_data, columns = ["Nb", "Cx", "Cy","Cz","Volume_unit"])
  Total_DF["Nb"] = Total_DF["Nb"] + 1
  Total_DF['IntDen_Total'] = pd.read_csv(nome_file_quantifiche_total_canale)['IntDen'].tolist()
  #Total_DF.drop(Total_DF[Total_DF['Volume_unit'] <= 10].index, inplace = True)
  Total_DF["Volume_pix"] = Total_DF["Volume_unit"] / voxel_size

  Nucleoli_DF = pd.DataFrame (nucleoli_data, columns = ["Nb", "Cx", "Cy","Cz","Volume_unit"])
  Nucleoli_DF["Nb"] = Nucleoli_DF["Nb"] + 1
  Nucleoli_DF['IntDen_Nucleoli'] = pd.read_csv(nome_file_quantifiche_nucleolo_canale)['IntDen'].tolist()
  #Nucleoli_DF.drop(Nucleoli_DF[Nucleoli_DF['Volume_unit'] >= 20].index, inplace = True)
  Nucleoli_DF["Volume_pix"] = Nucleoli_DF["Volume_unit"] / voxel_size

  conta_dxn = {}
  nuclei_index = []
  min_distance = []
  Nuclei_index_touse = Nuclei_DF.Nb.tolist()

  for i, row_D in Nucleoli_DF.iterrows():

    distances = []
    for k, row_N in Total_DF.iterrows():
      droplet_distance = distance_3D (row_D,row_N)
      distances.append(droplet_distance)

    index_min = distances.index(min(distances))
    min_distance.append(min(distances))
    nuclei_index.append(Nuclei_index_touse[index_min])


  Nucleoli_DF["Centroid_distance"] = min_distance
  Nucleoli_DF["Total_ID"] = nuclei_index
  Nucleoli_DF["Total_ID"] = Nucleoli_DF["Total_ID"]
  #FILTRI SUGLI OGGETTI
  Nucleoli_DF.drop(Nucleoli_DF[Nucleoli_DF['Centroid_distance'] >= 5].index, inplace = True)
  Nucleoli_DF.drop(Nucleoli_DF[Nucleoli_DF['Volume_unit'] <= 0.05].index, inplace = True)
  Nucleoli_DF.drop(Nucleoli_DF[Nucleoli_DF['Volume_unit'] >= 8].index, inplace = True)


  conta_dxn = Counter(Nucleoli_DF["Total_ID"])
  keys = list(conta_dxn.keys())
  keys.sort()
  sorted_conta_dxn = {i: conta_dxn[i] for i in keys}

  key_list = list(sorted_conta_dxn.keys())
  value_list = list(sorted_conta_dxn.values())
  Dots_Nucleus = pd.DataFrame(key_list, columns = ["Nucleo"])
  Dots_Nucleus["Dots"] = value_list

  Merged = pd.merge(Total_DF, Nucleoli_DF, left_on='Nb', right_on='Total_ID')
  Merged["Distanza norm."] = Merged["Centroid_distance"]/volume_to_radius(Merged["Volume_unit_x"])*100
  Merged["nuclei_id"] = None
  Merged["nucleoli_numbers"] = None
  for i in range(0, len(list(conta_dxn.keys()))):
    Merged["nuclei_id"][i] = list(conta_dxn.keys())[i]
    Merged["nucleoli_numbers"][i] = list(conta_dxn.values())[i]
  Merged.to_excel(xlsx_file, sheet_name = "analysis")

  list_merged.append(Merged)
  xlsx_file.save()

dict_merged = {}
for i in range(0,len(list_merged),2):
  dict_merged[list_merged[i]] = list_merged[i+1]

xlsx_file_path_summary = "/content/drive/MyDrive/Analysis/" + "summary.xlsx"
xlsx_file_summary = pd.ExcelWriter(xlsx_file_path_summary, engine='xlsxwriter')


easy_plot_dn = None
keys = []
for key in dict_merged.keys():
  easy_plot_dn = pd.concat([easy_plot_dn, dict_merged[key]["Distanza norm."]], axis = 1)
  keys.append(key)
easy_plot_dn.columns = keys
easy_plot_dn.to_excel(xlsx_file_summary, sheet_name = "distanza_normalizzata")

easy_plot_vol = None
keys = []
for key in dict_merged.keys():
  easy_plot_vol = pd.concat([easy_plot_vol, dict_merged[key]["Volume_unit_y"]], axis = 1)
  keys.append(key)
easy_plot_vol.columns = keys
easy_plot_vol.to_excel(xlsx_file_summary, sheet_name = "Volumi")

easy_plot_nnuc = None
keys = []
for key in dict_merged.keys():
  easy_plot_nnuc = pd.concat([easy_plot_nnuc, dict_merged[key]["nucleoli_numbers"]], axis = 1)
  keys.append(key)
easy_plot_nnuc.columns = keys
easy_plot_nnuc.to_excel(xlsx_file_summary, sheet_name = "Numero nucleoli")

xlsx_file_summary.save()


In [ ]:
Nuclei_DF

In [ ]:
easy_plot_ = pd.DataFrame(columns = dict_merged.keys())

for key in dict_merged.keys():
  easy_plot[key] = dict_merged[key]["Distanza norm."]

In [ ]:
dict_merged.keys()

In [ ]:
x

In [ ]:
easy_plot_dn = pd.DataFrame(columns = dict_merged.keys())
for key in dict_merged.keys():
  easy_plot_dn[key] = dict_merged[key]["Distanza norm."]
easy_plot_dn.to_excel(xlsx_file_summary, sheet_name = "distanza_normalizzata")

In [ ]:
easy_plot_dn

In [ ]:
concats = None
keys = []
for key in dict_merged.keys():
  dict_merged[key].rename(columns={"Distanza norm":key})
  concats = pd.concat([concats, dict_merged[key]["Distanza norm."]], axis = 1)
  keys.append(key)
concats.columns = keys

In [ ]:
concats.columns = ['col1', 'col2', 'col3', 'col4','col1', 'col2', 'col3', 'col4','cols','cols']

In [ ]:
concats

In [ ]:
[2,3,4,5] * 2